<a href="https://colab.research.google.com/github/AdityaArakere/Sentiment-analysis-model/blob/main/TFModelWithFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import re
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

uploaded = files.upload()


Saving combined_dataset9.csv to combined_dataset9.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['combined_dataset9.csv']))
df = df[['text', 'sentiment']]

# Preprocess the dataset
# Convert sentiment labels to 0, 1, -1
# df['category'] = df['category'].map({-1: [1, 2], 0: [3, 0], 1: [4, 5]})

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].astype(str).apply(lambda x: x.translate(translator))

# Remove numbers
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove extra whitespaces
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

# Remove stopwords (optional)
# You can use NLTK or other libraries to remove stopwords if desired

# Print the preprocessed dataset
print(df.head())


                                                text  sentiment
0  i dont know how or why this film has a meager ...          1
1  for a long time it seemed like all the good ca...          1
2  terry gilliams and david peoples teamed up to ...          1
3  what is there to say about an antiestablishmen...          1
4  this movie was made only years after the end o...          1


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install transformers
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPool1D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00


In [ ]:
# Split the dataset into train and test sets
x = df['text'].values
y = df['sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=40000, test_size=8000, random_state=42)

# Print the sizes of the train and test sets
print("Train set size:", len(x_train))
print("Test set size:", len(x_test))
print(y_train[0:5])

Train set size: 40000
Test set size: 8000
[1 1 1 1 1]


In [ ]:
# Define a custom dataset class
class RedditDataset(tf.keras.utils.Sequence):
    def __init__(self, texts, labels, tokenizer, batch_size):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.on_epoch_end()

    def __len__(self):
        return len(self.texts) // self.batch_size

    def __getitem__(self, index):
        batch_texts = self.texts[index*self.batch_size:(index+1)*self.batch_size]
        batch_labels = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        batch_encodings = self.tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='tf'
        )

        return dict(batch_encodings), batch_labels

    def on_epoch_end(self):
        pass

# Set the hyperparameters
num_epochs = 6
batch_size = 16

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess the data
train_dataset = RedditDataset(x_train, y_train, tokenizer, batch_size=batch_size)
test_dataset = RedditDataset(x_test, y_test, tokenizer, batch_size=batch_size)

# Load the pre-trained BERT model for sentiment analysis
bertModel = TFBertModel.from_pretrained('bert-base-uncased')

# Get the BERT embeddings
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
embeddings = bertModel.bert(input_ids)[0]

# Apply global max pooling
out = tf.keras.layers.GlobalMaxPooling1D()(embeddings)

# Apply dense layers
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.5)(out)
out = Dense(32, activation='relu')(out)

# Create the final output layer
y = Dense(2, activation='softmax')(out)

# Create the model with the defined inputs and outputs
model = tf.keras.Model(inputs=input_ids, outputs=y)

# Make the desired layer trainable
model.layers[2].trainable = True

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=0.01, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['token_type_ids', 'attention_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2500/2500 [==============================] - 1208s 473ms/step - loss: 0.4611 - accuracy: 0.7786 - val_loss: 0.3669 - val_accuracy: 0.8418
Epoch 2/6
2500/2500 [==============================] - 1182s 473ms/step - loss: 0.3451 - accuracy: 0.8548 - val_loss: 0.3476 - val_accuracy: 0.8515
Epoch 3/6
2500/2500 [==============================] - 1190s 476ms/step - loss: 0.3189 - accuracy: 0.8704 - val_loss: 0.3440 - val_accuracy: 0.8579
Epoch 4/6
2500/2500 [==============================] - 1181s 473ms/step - loss: 0.3008 - accuracy: 0.8777 - val_loss: 0.3376 - val_accuracy: 0.8590
Epoch 5/6
2500/2500 [==============================] - 1189s 475ms/step - loss: 0.2881 - accuracy: 0.8837 - val_loss: 0.3359 - val_accuracy: 0.8633
Epoch 6/6
500/500 [==============================] - 76s 152ms/step - loss: 0.3354 - accuracy: 0.8646
Test Loss: 0.3354
Test Accuracy: 86.46%


In [ ]:
# Define the paths to save the model and tokenizer
model_path = '/content/drive/MyDrive/Models/tf_fine_tuned_model10'
tokenizer_path = '/content/drive/MyDrive/Models/tokenizer10'

# Save the fine-tuned model
model.save(model_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)


In [ ]:
from transformers import BertTokenizer

loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Models/tf_fine_tuned_model10")
# model.summary()
# loaded_model.summary()
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Models/tokenizer10')

# Example custom input
custom_input = "I am feeling very nice and happy right now"

# Tokenize the custom input
custom_input_encodings = tokenizer.encode_plus(
    custom_input,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

# Get the input IDs
input_ids = custom_input_encodings['input_ids']

# Get the predictions for the custom input
custom_predictions = loaded_model.predict({'input_ids': input_ids})

yprint(custom_predictions)

# Process the predictions
sentiment = "Positive" if custom_predictions[0][0] >= custom_predictions[0][1] else "Negative"
print(f"Input: {custom_input}")
print(f"Sentiment: {sentiment}")
